# Import Libraries

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
import torch.nn.functional as F
from fashion import fashion
from datetime import datetime
import time
from tqdm import tqdm
from torch.utils.data import DataLoader

ModuleNotFoundError: No module named 'fashion'

# Data : Extraction & Pre-processing

In [2]:

normalize = transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]],
                                     std=[x/255.0 for x in [63.0, 62.1, 66.7]])

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))])

train_dataset = fashion(root='data', 
                            train=True, 
                            transform=transform,
                            download=True
                           )

test_dataset = fashion(root='data', 
                            train=False, 
                            transform=transform,
                           )

# Data Loading

In [3]:
batch_size = 100
n_iters = 5500

train_loader = DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size,            
                                           shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

# Network Definition

In [4]:

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
     
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=0)
        self.relu2 = nn.ReLU()
        
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        self.dropout = nn.Dropout(p=0.5)

        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(32*4*4, 10)
    
    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        
        # Max pool 1
        out = self.maxpool1(out)
        
        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)
        
        # Max pool 2 
        out = self.maxpool2(out)
        
        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)
        out = self.dropout(out)
        # Linear function (readout)
        out = self.fc1(out)
        
        return out
    
    
model = CNNModel()

# Setting up trainer parameters

In [5]:
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train function

In [6]:
def train(network, epochs, train_loader, test_loader, criterion, optimizer,
              change_rate=None, use_gpu=False):
    print('\nTraining in progress...Time now: {}\n'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S')))
    
    try:

        if use_gpu:
            network = network.cuda()
            criterion = criterion.cuda()

        for epoch in range(epochs):
            epoch_time = time.time()
            print("--> Epoch: {}/{}".format(
                epoch,
                epochs - 1
            ))
            for mode in ['train', 'val']:
                if mode == 'train':
                    network.train()  # Set model to training mode
                    
                    loader = tqdm(train_loader, total=len(train_loader))
                    
                    for batch_idx, (data, target) in enumerate(loader):

                        data = Variable(data.float())
                        target = Variable(target)

                        if use_gpu:
                            data = data.cuda()
                            target = target.cuda()

                        # Forward + Backward + Optimize
                        optimizer.zero_grad()
                        output = network(data)
                        loss = nn.CrossEntropyLoss()(output, target)
                        loss.backward()
                        optimizer.step()
                    print('\tTrain set: Loss: {:.6f}'.format(loss.item()))

                else:
                    network.eval()  # Set model to evaluate mode
                    test_loss = 0
                    correct = 0
                    with torch.no_grad():
                        for data, target in test_loader:

                            data = Variable(data.float())
                            target = Variable(target)

                            if torch.cuda.is_available():
                                data = data.cuda()
                                target = target.cuda()

                            output = network(data)
                            test_loss += nn.CrossEntropyLoss()(output, target)
                            _, pred = torch.max(output.data, 1)
                            #pred = output.max(1)[1] # get the index of the max log-probability
                            correct += (pred == target).sum()

                    test_loss /= len(test_loader.dataset)
                    print('\n\tTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
                        test_loss, correct, len(test_loader.dataset),
                        100. * correct / len(test_loader.dataset)))
    except KeyboardInterrupt:
        print("\n\n**********Training stopped prematurely.**********\n\n")
    finally:
        print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    

# Training..

In [7]:
train(model, num_epochs, train_loader, test_loader, criterion, optimizer,
              change_rate=None, use_gpu=True)


Training in progress...Time now: 2018-07-26 22:17:14

--> Epoch: 0/8


100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [00:07<00:00, 78.62it/s]


	Train set: Loss: 0.383635

	Test set: Average loss: 0.0044, Accuracy: 8420/10000 (84%)

--> Epoch: 1/8


100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [00:06<00:00, 90.29it/s]


	Train set: Loss: 0.505372

	Test set: Average loss: 0.0040, Accuracy: 8625/10000 (86%)

--> Epoch: 2/8


...

	Train set: Loss: 0.289133

	Test set: Average loss: 0.0031, Accuracy: 8881/10000 (88%)

--> Epoch: 7/8


100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [00:06<00:00, 87.36it/s]


	Train set: Loss: 0.388536

	Test set: Average loss: 0.0030, Accuracy: 8923/10000 (89%)

--> Epoch: 8/8


100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [00:07<00:00, 78.98it/s]


	Train set: Loss: 0.356209

	Test set: Average loss: 0.0031, Accuracy: 8898/10000 (88%)

2018-07-26 22:18:28
